# Data Wrangling

https://www.cbp.gov/newsroom/stats

https://www.ice.gov/detain/detention-management

https://www.uscis.gov/tools/reports-and-studies/immigration-and-citizenship-data

https://ohss.dhs.gov/topics/immigration#other-resources 


# Ideas
- Write about H1B, H2A, H2B, or other visas
- Write about detained individual counts at the border
- Write about detained individuals within the US
- Forecast immigration data
- Forecast impacts of policy changes

- Combine company visa information with their stock information
- Investigate if there are any correlations between stock information and visa information
- Can we make price movement predictions with visa information of publicly listed companies



In [1]:
# Imports
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from yfinance import ticker
import json
from io import StringIO
import re

In [2]:
# Gets a list of all listed companies from SEC's EDGAR database
#"https://www.sec.gov/files/company_tickers.json"

with open('data/company_tickers.json', 'r') as f:
    data = json.load(f)
# companies = [entry["ticker"] for entry in response.values()]
# print(companies[:10])  # Print first 10 tickers

In [3]:
### More robust data wrangling pipeline once we get remove_meta_characters optimized / finish debugging files ###

# def read_tsv(file_path):
#     """Reads a TSV file with UTF-8 encoding."""
#     with open(file_path, "r", encoding="utf-8") as f:
#         return pd.read_csv(StringIO(f.read()), delimiter="\t")
    
# def remove_meta_characters(input_string):
#     if pd.isna(input_string) or not isinstance(input_string, str):
#         return ""
#     # Define a regex pattern to match all meta characters
#     cleaned_string = re.sub(r'^[A-Z0-9]+\s+', '', input_string)
#     cleaned_string = re.sub(r'[^\w\s]', '', cleaned_string)
#     return cleaned_string.strip()

# titles = [company['title'].upper() for company in data.values()]

# visa_files = {"h1b": ['data/h1b_2016_2009.csv', 'data/h1b_2024_2017.csv'],
#               "h2a": ['data/h2a_2019_2015.csv', 'data/h2a_2024_2020.csv'],
#               "h2b": ['data/h2b_2019_2015.csv', 'data/h2b_2025_2020.csv']}

# df_visas = []
# for visa_type, files in visa_files.items():
#     df_list = [read_tsv(file) for file in files]
#     df = pd.concat(df_list, axis=0)
#     df["employer"] = df["Employer (Petitioner) Name"].str.upper()#.apply(remove_meta_characters)
#     df_filtered = df[df["employer"].isin(titles)]
#     df_filtered["type"] = visa_type
#     df_visas.append(df_filtered)

# df_visas_combined = pd.concat(df_visas)

# drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Index()', 'Consular_processed', 'New Employment Denial', 'Continuation Denial', 'Change with Same Employer Denial', 'New Concurrent Denial', 'Change of Employer Denial', 'Amended Denial', 'Consular_Processed', 'ETA Case Number']
# df_visas_clean = df_visas_combined.drop(drop_cols, axis = 1)

# df_visas_clean.head()


In [4]:

h1b_2009_path = 'data/h1b_2016_2009.csv'
h1b_2017_path = 'data/h1b_2024_2017.csv'

h2a_2015_path = 'data/h2a_2019_2015.csv'
h2a_2020_path = 'data/h2a_2024_2020.csv'

h2b_2015_path = 'data/h2b_2019_2015.csv'
h2b_2020_path = 'data/h2b_2025_2020.csv'

#Had to convert all encoding to UTF8 via notepad++
def read_tsv(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return pd.read_csv(StringIO(f.read()), delimiter="\t")

# Read all files using the function
df_h1b_a = read_tsv(h1b_2009_path)
df_h1b_b = read_tsv(h1b_2017_path)

df_h2a_a = read_tsv(h2a_2015_path)
df_h2a_b = read_tsv(h2a_2020_path)

df_h2b_a = read_tsv(h2b_2015_path)
df_h2b_b = read_tsv(h2b_2020_path)

C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3014528854.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3014528854.py:13: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3014528854.py:13: DtypeWarning: Columns (4,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")
C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3014528854.py:13: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(StringIO(f.read()), delimiter="\t")


In [5]:
df_h1b = pd.concat([df_h1b_a, df_h1b_b], axis=0)

df_h2a = pd.concat([df_h2a_a, df_h2a_b], axis=0)

df_h2b = pd.concat([df_h2b_a, df_h2b_b], axis=0)


In [6]:
print(df_h1b.columns)
print(df_h2a.columns)
print(df_h2b.columns)

Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial'],
      dtype='object')
Index(['Index()', 'Action Fiscal Year', 'Employer (Petitioner) Name', 'Tax ID',
       'Industry', 'Occupation (SOC) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Worksite State',
       'Consular_processed', 'Wage Rate Band', 'New Employment Approval',
       'New Employment Denial', 'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial'],
      dtype='object')
Index(['Index()', 'Cap Fiscal Year', 'Cap Type', 'Employer (Petiti

In [7]:
titles = [company['title'].upper() for company in data.values()]

In [8]:
def remove_meta_characters(input_string):
    # Define a regex pattern to match all meta characters
    pattern = r'[^\w\s]'
    # Substitute meta characters with an empty string
    cleaned_string = re.sub(pattern, '', input_string)
    return cleaned_string

In [9]:
df_h1b['employer'] = df_h1b['Employer (Petitioner) Name'].str.upper()
df_h2a['employer'] = df_h2a['Employer (Petitioner) Name'].str.upper()
df_h2b['employer'] = df_h2b['Employer (Petitioner) Name'].str.upper()

In [10]:
# print(df_h2a[['Employer (Petitioner) Name', 'employer']].head(10))
print("Sample H2A Matches:", df_h2a[df_h2a["employer"].isin(titles)][["employer"]].head(2))
print("Sample H2B Matches:", df_h2b[df_h2b["employer"].isin(titles)][["employer"]].head(2))

Sample H2A Matches:      employer
3511  CHS INC
3512  CHS INC
Sample H2B Matches:                         employer
3876                     UTG INC
9245  GREEN LEAF INNOVATIONS INC


In [11]:
print('H1B Crossover Count ' + str(sum(df_h1b["employer"].isin(titles))))
print('H2A Crossover Count ' + str(sum(df_h2a["employer"].isin(titles))))
print('H2B Crossover Count ' + str(sum(df_h2b["employer"].isin(titles))))

H1B Crossover Count 5245
H2A Crossover Count 53
H2B Crossover Count 4


In [12]:
df_h1b_listed = df_h1b[ df_h1b["employer"].isin(titles) ]
df_h1b_listed['type'] = 'h1b'

df_h2a_listed = df_h2a[ df_h2a["employer"].isin(titles) ]
df_h2a_listed['type'] = 'h2a'

df_h2b_listed = df_h2b[ df_h2b["employer"].isin(titles) ]
df_h2b_listed['type'] = 'h2b'

C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3144496779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h1b_listed['type'] = 'h1b'
C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3144496779.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h2a_listed['type'] = 'h2a'
C:\Users\owenh\AppData\Local\Temp\ipykernel_12284\3144496779.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [13]:
print(df_h1b.columns)
print(df_h2a.columns)
print(df_h2b.columns)

Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial',
       'employer'],
      dtype='object')
Index(['Index()', 'Action Fiscal Year', 'Employer (Petitioner) Name', 'Tax ID',
       'Industry', 'Occupation (SOC) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Worksite State',
       'Consular_processed', 'Wage Rate Band', 'New Employment Approval',
       'New Employment Denial', 'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial',
       'employer'],
      dtype='object')
Index(['Index()', 'Cap Fisca

In [14]:
df_visas = pd.concat([df_h1b_listed, df_h2a_listed, df_h2b_listed])
df_visas.shape

(5302, 39)

h1b_drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Initial Approval', 'Continuing Approval']
#Adding the columns does not work since they are currently strings, to_numeric not working due to commas
df_h1b_listed['tot'] = df_h1b_listed['Initial Approval'] + df_h1b_listed['Continuing Approval']
df_h1b_clean = df_h1b_listed.drop(h1b_drop_cols, axis = 1)
df_h1b_clean

In [15]:
df_visas.columns


Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'Initial Approval',
       'Initial Denial', 'Continuing Approval', 'Continuing Denial',
       'employer', 'type', 'Index()', 'Action Fiscal Year', 'Industry',
       'Occupation (SOC) Code', 'Worksite State', 'Consular_processed',
       'Wage Rate Band', 'New Employment Approval', 'New Employment Denial',
       'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial',
       'Cap Fiscal Year', 'Cap Type', 'Work Site State', 'Consular_Processed',
       'Hourly Wage', 'ETA Case Number'],
      dtype='object')

In [16]:
drop_cols = ['Line by line', 'Employer (Petitioner) Name', 'Tax ID', 'Initial Denial', 'Continuing Denial', 'Index()', 'Consular_processed', 'New Employment Denial', 'Continuation Denial', 'Change with Same Employer Denial', 'New Concurrent Denial', 'Change of Employer Denial', 'Amended Denial', 'Consular_Processed', 'ETA Case Number']
df_visas_clean = df_visas.drop(drop_cols, axis = 1)

In [17]:
df_visas_clean.head()

,Fiscal Year,Industry (NAICS) Code,Petitioner City,Petitioner State,Petitioner Zip Code,Initial Approval,Continuing Approval,employer,type,Action Fiscal Year,...,New Employment Approval,Continuation Approval,Change with Same Employer Approval,New Concurrent Approval,Change of Employer Approval,Amended Approval,Cap Fiscal Year,Cap Type,Work Site State,Hourly Wage
547,2016.0,NaN,NORTH CHICAGO,IL,60064.0,1,1,ABBOTT LABORATORIES,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
548,2016.0,31-33 - Manufacturing,ABBOT PARK,IL,60064.0,0,1,ABBOTT LABORATORIES,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549,2016.0,31-33 - Manufacturing,ABBOTT PARK,IL,60064.0,2,41,ABBOTT LABORATORIES,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,2016.0,31-33 - Manufacturing,NORTH CHICAGO,IL,60064.0,0,5,ABBOTT LABORATORIES,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
694,2016.0,"54 - Professional, Scientific, and Technical S...",SAN DIEGO,CA,92130.0,1,1,ACADIA PHARMACEUTICALS INC,h1b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import time
def fetch_stock_data(tickers):
    all_data = {}
    print('Downloading yfinance historical price data 2000/01/01 to 2025/03/16')
    num_downloaded = 0
    for ticker in tickers:
        try:
            data = yf.download(ticker, start="2000-01-01", end="2025-03-16", progress = False)
            all_data[ticker] = data
            num_downloaded +=1
            time.sleep(2.2)  # Sleep for 2 seconds between requests to avoid being blocked
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
            time.sleep(2.2)

    print('Successfully downloaded ' + str(num_downloaded/len(tickers)))
    #Save a csv for offline work
    sp500_data = pd.concat(all_data, axis=1)
    sp500_data.to_csv("sp500_data.csv")
    
    print("S&P 500 historical data saved to sp500_data.csv")
    return sp500_data

#Only run this once on your system
#Make sure NOT to track the sp500_data csv
#sp500csv made for offline work
#fetch_stock_data